<a href="https://colab.research.google.com/github/yatta03/112-2PL/blob/main/project/med.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install pygsheets

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 51 n

In [ ]:
# import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
import pygsheets
import re

In [ ]:
class Crawler:
  def __init__(self):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument("--disable-gpu")  # Required for Windows
    chrome_options.add_argument('--no-sandbox')
    chrome_options.headless = True
    self.driver = webdriver.Chrome(options=chrome_options)

  # wait until locator presents and return parsed html of given url
  def getSoup(self, url, locator = None):
    self.driver.get(url)
    if(locator):
      try:
          WebDriverWait(self.driver, 10).until(EC.presence_of_element_located(locator), "not found")
      except:
          print("locator not found")
    else:
      self.driver.implicitly_wait(5)
    htmlContent = self.driver.page_source

    soup = bs(htmlContent, 'html.parser')
    return soup

  # quit the driver
  def quit(self):
    self.driver.quit()

# [Daiyu](http://www2.showbox.com.tw/Daiyu/pd_XiaoYan.htm)

In [ ]:
daiyu_url = "http://www2.showbox.com.tw/Daiyu/_bin/"
daiyu_base_url = "http://www2.showbox.com.tw/Daiyu/"

In [ ]:
crawler = Crawler()

In [ ]:
# test_url = 'http://www2.showbox.com.tw/Daiyu/pd_XiaoYan.htm'
def getMed(crawler, link):
  med = {}

  crawler.getSoup(link)

  frame2 = crawler.driver.find_element(By.NAME, 'mainFrame')
  crawler.driver.switch_to.frame(frame2)

  name = crawler.driver.find_element(By.CLASS_NAME, 'name1')
  name = re.sub(r'\n', '', name.text)
  img_path = crawler.driver.find_element(By.XPATH, "//img")
  app_txt = crawler.driver.find_element(By.CLASS_NAME, 'apptxt')
  app_txt = re.sub(r'\n', '', app_txt.text)
  tb_content = crawler.driver.find_elements(By.CLASS_NAME, 'td2')

  symptom = tb_content[0].text
  symptom = re.sub(r'[(（）)]', '', symptom)
  symptom_split = re.split(r"\n|、|。", symptom)
  symptom_split = [sym.strip() for sym in symptom_split if sym.strip()]

  ingredients = re.split(r"\n|、|。", tb_content[1].text)
  ingredients = [ing.strip() for ing in ingredients if ing.strip()]

  med["name"] = name
  med["img_path"] = img_path.get_attribute("src")
  med["app_txt"] = app_txt
  med["symptom"] = symptom_split
  med["ingredient"] = ingredients
  med["link"] = link

  return med

# getMed(crawler, test_url)

In [ ]:
locator = (By.ID, "body")
crawler.getSoup(daiyu_url, locator)

medicines = []

# switch to frame
frame = crawler.driver.find_element(By.NAME, 'body')
crawler.driver.switch_to.frame(frame)
frame2 = crawler.driver.find_element(By.NAME, 'mainFrame')
crawler.driver.switch_to.frame(frame2)
crawler.driver.get_screenshot_as_file("screenshot.png")

# get all medicine links
l = crawler.driver.find_elements(By.XPATH, "//*[@id='SE_List_All']/div/a")
links = [e.get_attribute("href") for e in l]

# get away duplicated one
links_set = set(links)
links = list(links_set)

# print(links)
for link in links:
  medicines.append(getMed(crawler, link))

In [ ]:
print(len(links))
daiyu_med = pd.DataFrame(medicines)
daiyu_med.head()

10


,name,img_path,app_txt,symptom,ingredient,link
0,正記加味消痔丸,http://www2.showbox.com.tw/Daiyu/pdsimg_XiaoYa...,衛署成製字第011802號,"[內痔, 外痔, 大便祕結, 止血消腫]","[槐花, 生地黃, 黃連, 甘草, 黃柏, 山梔子, 地榆, 知母, 黃芩, 大黃, 側柏葉...",http://www2.showbox.com.tw/Daiyu/pd_SJWplus.htm
1,三支雨傘標友露安,http://www2.showbox.com.tw/Daiyu/pdsimg_XiaoYa...,衛署藥製字第045350號G-7852,"[緩解感冒之各種症狀, 流鼻水, 鼻塞, 打噴嚏, 咽喉痛, 咳嗽, 咳痰, 畏寒, 發燒,...","[Acetaminophen., dl-Methylephedrine hydrochlor...",http://www2.showbox.com.tw/Daiyu/pd_YLA.htm
2,友露安感冒膜衣錠,http://www2.showbox.com.tw/Daiyu/pdsimg_XiaoYa...,衛署藥製字第048791號GMP G-9638,"[緩解感冒之各種症狀, 頭痛, 發燒, 流鼻水, 鼻塞, 畏寒, 打噴嚏, 咳嗽, 喀痰, ...","[Acetaminophen, Potassium guaiacol sulfonate, ...",http://www2.showbox.com.tw/Daiyu/pd_YLA_pill.htm
3,"""大裕""消炎止痛膜衣錠200毫克",http://www2.showbox.com.tw/Daiyu/pdsimg_XiaoYa...,衛署藥製字第052594號 GMP G-10928,"[暫時性解除頭痛, 肌肉疼痛, 關節炎的輕度疼痛, 牙痛, 輕度疼痛及與感冒有關的疼痛, 經...",[Ibuprofen 200 mg],http://www2.showbox.com.tw/Daiyu/pd_XiaoYan.htm
4,剋百痛貼布,http://www2.showbox.com.tw/Daiyu/pdsimg_XiaoYa...,衛署成製字第011644號G-7345,"[腰痛, 打撲, 捻挫, 肩膀酸痛, 關節痛, 肌肉痛, 肌肉疲勞]","[Dl-Camphor., L-Menthol., Methyl Salicylate.]",http://www2.showbox.com.tw/Daiyu/pd_KPT.htm


In [ ]:
crawler.quit()

In [ ]:
#authorization
gc = pygsheets.authorize(service_file='./pl-421203-a74b8641e6bc.json')
sheetUrl = "https://docs.google.com/spreadsheets/d/16_34--12pFGZ3alnL_mAuCEE5ne0aaxZ0sd8qlJzyOo/edit?usp=sharing"
sh = gc.open_by_url(sheetUrl)

In [ ]:
def dfToSheet(df, sh, title):
  try:
    ws = sh.worksheet_by_title(title)
    ws.set_dataframe(df, start=(0,0), nan='')
  except Exception as e:
    print(e)
def clearSheet(sh, title):
  try:
    ws = sh.worksheet_by_title(title)
    ws.clear()
  except Exception as e:
    print(e)

In [ ]:
clearSheet(sh, "Daiyu")
dfToSheet(daiyu_med, sh, "Daiyu")

# this part is for [ysphealth](https://www.ysphealth.com/cate-167431.htm)

In [ ]:
crawler = Crawler()

In [ ]:
ysp_url = "https://www.ysphealth.com/cate-162368.htm"

In [ ]:
links = []
page = 1
while True:
  crawler.getSoup(ysp_url + "?page=" + str(page))
  products = crawler.driver.find_elements(By.CLASS_NAME, "product-col")
  if not products:
    break
  for p in products:
    link = p.find_element(By.TAG_NAME, "a")
    # print(link.get_attribute("href"))
    links.append(link.get_attribute("href"))
  page += 1

In [ ]:
print(len(links))
print(links)

41
['https://www.ysphealth.com/10-3851202.html', 'https://www.ysphealth.com/https-www-ysphealth-com-fucole-suppositories-html.html', 'https://www.ysphealth.com/voren-patch.html', 'https://www.ysphealth.com/https-www-ysphealth-com-paran-tablets-html.html', 'https://www.ysphealth.com/https-www-ysphealth-com-show-417315-html.html', 'https://www.ysphealth.com/https-www-ysphealth-com-show-417333-html.html', 'https://www.ysphealth.com/eapilone-tablets.html', 'https://www.ysphealth.com/https-www-ysphealth-com-winsolve-sulnafine-cream-html.html', 'https://www.ysphealth.com/https-www-ysphealth-com-winsolve-sulnafine-spray-html.html', 'https://www.ysphealth.com/https-www-ysphealth-com-winsolve-comesnail-solution-html.html', 'https://www.ysphealth.com/show-1112344.html', 'https://www.ysphealth.com/https-www-ysphealth-com-antifungal-cream-html.html', 'https://www.ysphealth.com/https-www-ysphealth-com-winsolve-bumin-ointment-html.html', 'https://www.ysphealth.com/show-1128128.html', 'https://www.ys

In [ ]:
test_url = 'https://www.ysphealth.com/10-3851202.html'

In [ ]:
def getMed_ysp(crawler, link):
  med = {}
  locator = (By.CLASS_NAME, "product-info")
  crawler.getSoup(link, locator)
  img = crawler.driver.find_element(By.CLASS_NAME, "gc-display-display")
  img_path = img.get_attribute("src")
  name = img.get_attribute("alt")

  content_block = crawler.driver.find_element(By.ID, "content_tab")
  # contents = content_block.find_elements(By.TAG_NAME, "p")
  contents = re.sub(r'\n', ' ', content_block.text)
  # print(contents)

  p1 = r'適應症:(.*?)成分:'
  p2 = r'\b衛部藥.*'
  # Find the substrings using regular expressions
  match1 = re.search(p1, contents, re.DOTALL)
  match2 = re.search(p2, contents, re.DOTALL)

  # Extract the substrings
  symptoms = match1.group(1).strip() if match1 else ""
  app_txt = match2[0] if match2 else ""

  content_break = content_block.text.split("\n")
  ingredients = []
  for con in content_break:
    match3 = re.search(r'.*\.', con)
    if match3:
      ing = re.sub("\.", "", match3[0])
      ingredients.append(ing)
  # print(symptoms, ingredients, app_txt)

  med["name"] = name
  med["img_path"] = img_path
  med["app_txt"] = app_txt
  med["symptom"] = symptoms
  med["ingredient"] = ingredients
  med["link"] = link

  return med
# getMed_ysp(crawler, test_url)

In [ ]:
ysh_medicines = []
for link in links:
  ysh_medicines.append(getMed_ysp(crawler, link))

In [ ]:
crawler.quit()

In [ ]:
ysh_medicines_df = pd.DataFrame(ysh_medicines)
ysh_medicines_df.head()

,name,img_path,app_txt,symptom,ingredient,link
0,理冒 喉悅喉錠10錠入,https://img.mweb.com.tw/thumb/626/1000x1000/pr...,衛部藥製字第 059827 號,喉嚨發炎引起的沙啞，喉嚨乾澀，口腔內的殺菌和消毒。,"[Chlorhexidine HCL, Glycyrrhiza Extract]",https://www.ysphealth.com/10-3851202.html
1,理冒伯樂小兒用栓劑125毫克,https://img.mweb.com.tw/thumb/626/1000x1000/pr...,衛部藥輸字第028169號,退燒、止痛（緩解頭痛、牙痛、咽喉痛、關節痛、神經痛、肌肉酸痛、月經痛)。,[每粒中含：Acetaminophen],https://www.ysphealth.com/https-www-ysphealth-...
2,非炎-酸痛水性貼布,https://img.mweb.com.tw/thumb/626/1000x1000/pr...,,短期使用以緩解因發炎反應引起之局部疼痛。,"[每片12g(100cm×14, Diclofenac Sodium]",https://www.ysphealth.com/voren-patch.html
3,理冒-伯樂止痛錠,https://img.mweb.com.tw/thumb/626/1000x1000/pr...,,退燒、止痛（緩解頭痛、牙痛、咽喉痛、關節痛、神經痛、肌肉酸痛、月經痛）。,[每錠中含：Acetaminophen],https://www.ysphealth.com/https-www-ysphealth-...
4,理冒-喉悅喉錠20錠入,https://img.mweb.com.tw/thumb/626/1000x1000/pr...,衛部藥製字第 059827 號 以上內容為經行政院衛生福利部核准之仿單,喉嚨發炎引起的沙啞，喉嚨乾澀，口腔內的殺菌和消毒。,"[Chlorhexidine HCL, Glycyrrhiza Extract]",https://www.ysphealth.com/https-www-ysphealth-...


In [ ]:
clearSheet(sh, "ysh")
dfToSheet(ysh_medicines_df, sh, "ysh")